In [ ]:
!pip install --upgrade plotly

     |████████████████████████████████| 12.9MB 7.2MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import xgboost as xgb
from fbprophet import Prophet

from google.colab import drive
drive.mount('/content/gdrive')

os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Covid19Hackathon"

from sklearn.metrics import mean_squared_error
from math import sqrt
# pd.options.plotting.backend = "plotly"

pd.set_option('display.max_rows', 750)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)
pd.set_option('display.max_colwidth', 200)

from IPython.display import display

%cd /content/gdrive/My Drive/Covid19Hackathon

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Covid19Hackathon


In [ ]:
indv_data_list = [pd.read_csv("https://api.covid19india.org/csv/latest/raw_data" + str(i+1) + ".csv") for i in range(12)]
indv_data = pd.concat(indv_data_list, axis = 0).reset_index(drop = True)
state_code_mapping = (indv_data['State code'] + "-" + indv_data['Detected State']).dropna(axis = 0).unique()
state_code_mapping = {i.split("-")[0]:i.split("-")[1] for i in list(state_code_mapping)}

In [ ]:
def state_data_vivek(df):
  state_wise_daily = df
  base_data_df = pd.DataFrame()
  base_data_df['Date'] = state_wise_daily.Date_time.unique()
  base_data_df['Key'] = 1
  # base_data_df['Year'] = base_data_df['Date'].apply(lambda x: x.strftime('%d-%m-%Y'))
  print(base_data_df.head(5))
  print(base_data_df.shape)
  list_states = state_wise_daily.columns.tolist()
  rm_ilst = ['Date', 'Status', 'TT', 'Date_time']
  for i in rm_ilst:
      list_states.remove(i)#,"Status","Date_time","TT"])
  state_df = pd.DataFrame()
  state_df["State_cd"] = list_states
  state_df["Key"] = 1

  base_data_df = base_data_df.merge(state_df, on='Key', how='left')
  base_data_df = base_data_df.drop(columns = ['Key'])
  print(base_data_df.shape)
  base_data_df.head(2)

  conf_list = []
  dead_lsit = []
  recv_list = []
  for i in range(0, len(base_data_df['Date'])):
      val = 0
      val = state_wise_daily[(state_wise_daily['Date_time'] == base_data_df['Date'][i]) & (state_wise_daily['Status'] == 'Confirmed')][base_data_df['State_cd'][i]].values[0]
      conf_list.append(val)
  for i in range(0, len(base_data_df['Date'])):
      val = 0
      val = state_wise_daily[(state_wise_daily['Date_time'] == base_data_df['Date'][i]) & (state_wise_daily['Status'] == 'Deceased')][base_data_df['State_cd'][i]].values[0]
      dead_lsit.append(val)
  for i in range(0, len(base_data_df['Date'])):
      val = 0
      val = state_wise_daily[(state_wise_daily['Date_time'] == base_data_df['Date'][i]) & (state_wise_daily['Status'] == 'Recovered')][base_data_df['State_cd'][i]].values[0]
      recv_list.append(val)
  base_data_df['Confirmed'] = conf_list
  base_data_df['Deceased']  = dead_lsit
  base_data_df['Recovered'] = recv_list
  return(base_data_df)


# Module for creating time-series variables

def ts_features(data_, n, widths = [7,14,21]): # ndays behind 21 days ## data contains DatexState Cases # State = [] means by default it predicts for Overall

  data = data_.copy() 
  data.set_index(['Date'], inplace = True)  

  merged = data.copy()
  for i in range(1,n):
    df =  data.copy()
    df = df.shift(i)      # shift with lag step=1
    df.columns = [j.replace(str(1),str(i+1)) for j in df.columns]
    merged = merged.merge(df, how = 'left', left_index=True, right_index=True)

  for width in widths:
    df =  data.copy()
    df = df.rolling(width).mean()      # pull out the State, shift with lag step=1, roll
    df.columns = [j.replace("_lag1", "") + "_rolling_last" + str(width) for j in df.columns]
    merged = merged.merge(df, how = 'left', left_index=True, right_index=True)

  merged = merged.reset_index()
  merged['dayofweek'] = merged['Date'].dt.dayofweek
  merged['isweekend'] = np.where(merged['dayofweek'].isin([5,6]), 1, 00)

  return(merged)

# def ts_features(data_, n, widths = [7,14,21]): # ndays behind 21 days ## data contains DatexState Cases # State = [] means by default it predicts for Overall

#   data = data_.copy()
#   data.set_index(['Date','State'], inplace = True)  

#   merged = data.copy()
#   for i in range(1,n+1):
#     df =  data.copy()
#     df = df.unstack().shift(i)      # pull out the State, shift with lag step=1
#     df = df.stack(dropna=False)      # stack the State back, keep the missing values
#     df.columns = [j + "_lag" + str(i) for j in df.columns]
#     merged = merged.merge(df, how = 'left', left_index=True, right_index=True)

#   for width in widths:
#     df =  data.copy()
#     df = df.unstack().shift(1).rolling(width).mean()      # pull out the State, shift with lag step=1, roll
#     df = df.stack(dropna=False)      # stack the State back, keep the missing values
#     df.columns = [j + "_rolling_last" + str(width) for j in df.columns]
#     merged = merged.merge(df, how = 'left', left_index=True, right_index=True)

#   merged = merged.reset_index()
#   merged['dayofweek'] = merged['Date'].dt.dayofweek
#   merged['isweekend'] = np.where(merged['dayofweek'].isin([5,6]), 1, 00)

#   return(merged)

# Data Prep

In [ ]:
_state_daily = pd.read_csv("https://api.covid19india.org/csv/latest/state_wise_daily.csv")
_state_daily['Date_time'] = pd.to_datetime(_state_daily['Date'] + "20", format = "%d-%b-%Y")
_state_daily.rename(state_code_mapping, axis = 1, inplace = True)
_state_daily.drop('DD', axis = 1, inplace = True)

state_daily = state_data_vivek(_state_daily).rename({'State_cd': 'State'}, axis = 1)


# Adding Testing Data (to be removed later/ cummax to be taken in model/logistic growth)
state_test_daily = pd.read_csv('https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv')
print(state_test_daily.shape)
# Select & Rename
state_test_daily = state_test_daily[['Updated On', 'State', 'Total Tested', 'Positive']].rename(columns={'Updated On' : 'Date', 'State' : 'State', 'Total Tested' : 'Tests', 'Positive' : 'Cases' })
# state_test_daily = state_test_daily[['Updated On', 'State', 'Test positivity rate']].rename(columns={'Updated On' : 'Date', 'State' : 'State', 'Test positivity rate': 'Positivity Rate' })
print(state_test_daily.shape)

# Add Date Mapping
state_test_daily['Date'] = pd.to_datetime(state_test_daily['Date'], format = '%d/%m/%Y')
state_test_daily = state_test_daily.drop_duplicates()
print(state_test_daily.shape)
state_test_daily.head(2)

# base_data = pd.read_csv(os.getcwd() + "/XgBoost/XgbData.csv")
# base_data['Date'] = pd.to_datetime(base_data['Date'],format = "%Y-%m-%d")
# base_data = base_data[['Date', 'State', 'Confirmed']]

base_data = state_daily[['Date', 'State', 'Confirmed']]
base_data = base_data.merge(state_test_daily, how = 'left', on = ['Date', 'State'])

display(base_data.head())

        Date  Key
0 2020-03-14    1
1 2020-03-15    1
2 2020-03-16    1
3 2020-03-17    1
4 2020-03-18    1
(146, 2)
(5402, 2)
(3967, 28)
(3967, 4)
(3966, 4)


,Date,State,Confirmed,Tests,Cases
0,2020-03-14,Andaman and Nicobar Islands,0,NaN,NaN
1,2020-03-14,Andhra Pradesh,1,NaN,NaN
2,2020-03-14,Arunachal Pradesh,0,NaN,NaN
3,2020-03-14,Assam,0,NaN,NaN
4,2020-03-14,Bihar,0,NaN,NaN


In [ ]:
st_list = ['Maharashtra', 'Tamil Nadu', 'Delhi', 'Karnataka', 'Andhra Pradesh', 'Uttar Pradesh', 'West Bengal', 'Gujarat', 'Telangana', 'Rajasthan', 'Bihar', 'Assam', 'Haryana', 'Madhya Pradesh', 'Odisha', 'Kerala', 'Jammu and Kashmir', 'Punjab', 'Jharkhand', 'Chhattisgarh', 'Uttarakhand', 'Goa', 'Tripura', 'Puducherry', 'Manipur', 'Himachal Pradesh', 'Ladakh', 'Nagaland', 'Arunachal Pradesh', 'Dadra and Nagar Haveli and Daman and Diu', 'Chandigarh', 'Meghalaya', 'Sikkim', 'Mizoram', 'Andaman and Nicobar Islands']
cnf_list = [10483, 5880, 1192, 6670, 10171, 4404, 2954, 1034, 2207, 422, 3646, 2372, 751, 734, 1833, 1251, 473, 1035, 708, 523, 369, 333, 128, 241, 124, 28, 3, 77, 93, 44, 57, 51, 29, 21, 96]
Aug7 = pd.DataFrame(zip(st_list, cnf_list), columns = ['State', 'Confirmed'])
Aug7['Date'] = pd.Timestamp('2020-08-07')
base_data = base_data.append(Aug7)

# Construct 1

One-Step prediction added to next train

In [ ]:
# def metrics(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
    
#     print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)) )
#     print("MAPE:", np.mean(np.abs((y_true - y_pred) / y_true)) * 100)
#       # perf = pd.DataFrame({'State': [state], 'RMSE': [rmse], 'MAPE': [mape]})
#     print(f"{state}\t{sum(y_true)}\t{rmse}\t{mape}")
#     return(perf)

def metrics(state, y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    perf = pd.DataFrame({'State': [state], 'RMSE': [rmse], 'MAPE': [mape]})
    print(f"{state}\t{sum(y_true)}\t{rmse}\t{mape}")
    return(perf)

def model_object1(growth_type): #Prophet Model

  prophet = Prophet(growth=growth_type, daily_seasonality= False, yearly_seasonality= False)# ,changepoints = [pd.Timestamp("2020-06-20 00:00:00")])
  return(prophet)

def prophet_1_day_prediction(model, train, test):

  model.fit(train)
  forecast = model.predict(test.drop('y', axis = 1))

  # Appending the new prediction to train data (Only Base Features i.e. ds, y_hat)
  new_prediction = forecast[['ds', 'yhat']].rename({'yhat': 'y'}, axis = 1)

  train = train[['ds', 'y']]
  updated_train = pd.concat([train, new_prediction], axis = 0).reset_index(drop = True)
  updated_train.rename({'ds':'Date', 'y': 'Confirmed'}, axis = 1, inplace = True)

  new_prediction = new_prediction.rename({'y': 'pred'}, axis = 1)

  # Returning the updated train data for deriving features
  return(updated_train, new_prediction)


# trend + seasonality + holidays
# Need to pass no. of tests in some way
# Come up with better missing value_imputation #cummax yet
def model_run(data, start_date, end_date, state, days = 1, growth_type = 'linear'):

  model =  model_object1(growth_type)#call the model function
  
  # This Below Data Contains Data Till Training Start Date with Date x State x Time 
  data1 = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)].copy() # 1 day extra for the single step test features [first prediction]

  preds = pd.DataFrame()
  for day in range(days):
    ## Creating a 1 day lagged variable to further derive other features later
    data1.set_index(['Date'], inplace = True)
    df =  data1.copy()
    df = df.shift(1)      # pull out the State, shift with lag step=1
    df.columns = [j + "_lag" + str(1) for j in df.columns]
    data1 = data1.merge(df, how = 'left', left_index=True, right_index=True)
    data1.reset_index(inplace = True)

    # Extracting actuals to be merged after creating lagged features
    actuals = data1[['Date', 'Confirmed']]

    data1.drop(list(set(['Tests',	'Cases',	'Confirmed']) & set(data1.columns)), axis = 1, inplace = True)
    
    # Adding one row for the single step test data
    last_date = data1['Date'].max()
    data1.set_index('Date', inplace = True)
    data1.loc[last_date + pd.Timedelta(1, 'D')] = data1.loc[last_date, :]
    data1.reset_index(inplace = True)

    ### Pass the above train df to module creating time-series features [21 days lags, moving averages, etc]
    data2 = ts_features(data1, 7, [7,14,21])
    data2 = data2.merge(actuals, how = 'left', on = 'Date').rename({'Confirmed':'y', 'Date': 'ds'}, axis = 1)#.drop('Positivity Rate', axis = 1)

    train = data2[(data2['ds'] >= start_date ) & (data2['ds'] <= (end_date + pd.Timedelta(day, 'D')))]
    test = data2[data2['ds'] > (end_date + pd.Timedelta(day, 'D'))]
    # test = pd.DataFrame({'ds': [(end_date + pd.Timedelta(day + 1, 'D'))], 'y':[np.nan]})

    # Missing Value Imputation
    train.fillna(0, inplace = True)
    test.fillna(0, inplace = True)

    # Initiating Model Object
    prophet = Prophet(growth=growth_type, daily_seasonality= False, yearly_seasonality= False)# ,changepoints = [pd.Timestamp("2020-06-20 00:00:00")])

    # Specifying whether to use Regressors or not
    regressors = [i for i in train.columns.tolist() if i!= 'ds' and i!='y']
  
    if len(regressors) > 0:
      for i in range(1, len(regressors)):
          prophet.add_regressor(regressors[i])

    # Specifying Floor and Caps for the Logistic Growth
    train['floor'] = 0
    test['floor'] = 0
    if growth_type == 'logistic':
      train['cap'] = 1 + train['y'].quantile([0.99]).values[0]
      test['cap'] = train['cap'].max()

    # regressors = [i for i in test.columns if y in i and i!=y]
    data1, pred = prophet_1_day_prediction(prophet, train, test)
    preds = preds.append(pred)

  preds['pred'] = preds['pred'].astype(int)

  preds = preds.rename({'ds':'Date'}, axis = 1).merge(data[['Date', 'Confirmed']], how = 'left', on = 'Date')

  perf = pd.DataFrame()#metrics(state, preds['Confirmed'], preds['pred'])
  perf['Date'] = start_date
  return(preds, perf)

In [ ]:
states = ["Maharashtra", "Tamil Nadu", "Delhi", "Karnataka", "Andhra Pradesh", "Uttar Pradesh", "West Bengal", "Gujarat", "Telangana", "Rajasthan", "Bihar", "Assam", "Haryana", "Madhya Pradesh", "Odisha", "Kerala", "Jammu and Kashmir", "Punjab", "Jharkhand", "Chhattisgarh", "Uttarakhand", "Goa", "Tripura", "Puducherry", "Manipur", "Himachal Pradesh", "Ladakh", "Nagaland", "Arunachal Pradesh", "Dadra and Nagar Haveli and Daman and Diu", "Chandigarh", "Meghalaya", "Sikkim", "Mizoram", "Andaman and Nicobar Islands"]

group_1 = ["Karnataka", "Telangana", "Andhra Pradesh", "Dadra and Nagar Haveli and Daman and Diu", "Puducherry", "Assam", "Odisha", "Goa", "Arunachal Pradesh", "Kerala"]
group_2 = ["Nagaland", "Sikkim", "Tamil Nadu", "West Bengal", "Uttar Pradesh", "Manipur", "Bihar", "Haryana", "Jharkhand", "Mizoram", "Chhattisgarh", "Punjab", "Maharashtra", "Delhi", "Tripura", "Jammu and Kashmir"]
group_3 = ["Himachal Pradesh", "Ladakh", "Rajasthan", "Uttarakhand"]
group_4 = ["Gujarat", "Madhya Pradesh", "Meghalaya", "Chandigarh"]

final_states_one_step_10thJul = ["Maharashtra", "Uttar Pradesh", "West Bengal", "Bihar", "Madhya Pradesh", "Kerala", "Uttarakhand"]
final_states_for_17thJul = ["Maharashtra", "Uttar Pradesh", "West Bengal", "Kerala", "Uttarakhand"] 

In [ ]:
base_data[base_data['State'].isin(group_1)].groupby('Date').sum().tail()

,Confirmed,Tests,Cases
Date,,,
2020-08-03,18830,6753309.0,496084.0
2020-08-04,23144,6934256.0,517920.0
2020-08-05,23319,7164394.0,541194.0
2020-08-06,25109,7399201.0,566260.0
2020-08-07,25215,0.0,0.0


In [ ]:
# pred_df = []
# conf_df = []
# for state in states:
#   print(state)
#   model_data = base_data[base_data['State'] == state].drop('State', axis = 1)
#   # display(model_data.set_index('Date')['Confirmed'].plot())

#   predictions = model_run(model_data, pd.Timestamp("2020-04-08 00:00:00"), pd.Timestamp("2020-07-10 00:00:00"), 15)
#   pred_df = pred_df + [predictions[['pred']].rename({'pred':state}, axis = 1)]
#   conf_df = conf_df + [predictions[['Confirmed']].rename({'Confirmed':state}, axis = 1)]

In [ ]:
# for i in range(len(states)):
#   print(i, states[i])

In [ ]:
final_states_for_17thJul_dates = {"Maharashtra":pd.Timestamp("5-8-2020 00:00:00"),
"Uttar Pradesh" :pd.Timestamp("5-1-2020 00:00:00"),
"West Bengal" :pd.Timestamp("5-15-2020 00:00:00"),
"Kerala" :pd.Timestamp("6-19-2020 00:00:00"),
"Uttarakhand" :pd.Timestamp("5-1-2020 00:00:00"),
}
last_date = pd.Timestamp("8-06-2020 00:00:00")

# final_states_for_17thJul_dates = {"Maharashtra": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_states_for_17thJul_dates["Maharashtra"]) ,
# "Uttar Pradesh": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_states_for_17thJul_dates["Uttar Pradesh"]) ,
# "West Bengal": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_states_for_17thJul_dates["West Bengal"]) ,
# "Kerala": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_states_for_17thJul_dates["Kerala"]),
# "Uttarakhand": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_states_for_17thJul_dates["Uttarakhand"])}


In [ ]:
pred_df = []
conf_df = []
i = 1
perf_df = pd.DataFrame()

for state in final_states_for_17thJul:#final_states_one_step_10thJul:#states[0:1]:

  for day in [0, 7, 14, 21, 28, 35, 42, 49, 56][0:1]:
    start_date = final_states_for_17thJul_dates[state]#pd.Timestamp("2020-05-01 00:00:00") + pd.Timedelta(day, 'd') #opt_dates_10thJuly[state] ##
    end_date = pd.Timestamp("2020-08-07 00:00:00")
  
    model_data = base_data[base_data['State'].isin([state])].drop('State', axis = 1)#.groupby('Date').sum().reset_index()
    predictions, perf = model_run(model_data, start_date, end_date, state,  21, 'linear' )
    perf_df = perf_df.append(perf)

      # display(model_data.set_index('Date')['Confirmed'].plot())
    pred_df = pred_df + [predictions[['pred']].rename({'pred':state}, axis = 1)]
    conf_df = conf_df + [predictions[['Confirmed']].rename({'Confirmed':state}, axis = 1)]
    i = i + 1

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:96: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [ ]:
display(pd.concat(pred_df, axis = 1))
display(pd.concat(conf_df, axis = 1))

,Maharashtra,Uttar Pradesh,West Bengal,Kerala,Uttarakhand
0,10119,3077,5287,2244,358
1,10814,5193,3478,1433,387
2,9761,4183,6146,1521,359
3,9416,3481,3706,1799,413
4,11077,4337,7317,1668,464
5,11773,5021,3264,1472,463
6,11386,3402,9869,1979,497
7,11495,4103,1668,1573,497
8,11789,5024,13759,1692,495
9,10866,3751,-1221,1708,476


,Maharashtra,Uttar Pradesh,West Bengal,Kerala,Uttarakhand
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


# Grouped Predictions

In [ ]:
pred_df = []
conf_df = []
i = 1

for group in (group_1, group_2, group_3, group_4):
  model_data = base_data[base_data['State'].isin(group)].groupby('Date').sum().reset_index()
  # display(model_data.set_index('Date')['Confirmed'].plot())

  predictions = model_run(model_data, pd.Timestamp("2020-04-08 00:00:00"), pd.Timestamp("2020-07-10 00:00:00"), 15)
  pred_df = pred_df + [predictions[['pred']].rename({'pred':"group_"+str(i)}, axis = 1)]
  conf_df = conf_df + [predictions[['Confirmed']].rename({'Confirmed':"group_"+str(i)}, axis = 1)]
  i = i + 1

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

RMSE: 5732.327380741613
MAPE: 28.4703643605008


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

RMSE: 2865.3675854940498
MAPE: 9.592449245987806


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

RMSE: 527.0072105768572
MAPE: 39.26308723904112


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

RMSE: 430.0571279880539
MAPE: 22.83979339120129


In [ ]:
pd.concat(pred_df, axis = 1)

,group_1,group_2,group_3,group_4
0,7810,18848,829,1421
1,8069,18155,626,1251
2,8011,17867,671,1288
3,8319,18379,686,1282
4,8241,19257,619,1317
5,8556,20243,569,1300
6,8508,20638,595,1264
7,8807,20777,571,1358
8,8392,20910,584,1276
9,8624,20426,551,1294


In [ ]:
pd.concat(conf_df, axis = 1)

,group_1,group_2,group_3,group_4
0,7584,18771,642,1505
1,7795,19539,804,1353
2,8521,18141,652,1527
3,8270,19280,814,1727
4,9843,20802,1011,1601
5,10920,22098,975,1710
6,10883,22208,779,1704
7,12409,22746,933,1687
8,13441,23826,1256,1860
9,11840,22005,1210,1747


# Forecasting

In [ ]:
model_data = base_data[base_data['State'] == 'Maharashtra'].drop('State', axis = 1)
model_data = model_data[(model_data['Date'] >= pd.Timestamp("2020-04-08 00:00:00")) & (model_data['Date'] <= pd.Timestamp("2020-06-28 00:00:00"))]
# model_data.set_index('Date')['Confirmed'].plot()

prophet = Prophet(growth='linear', daily_seasonality= False, yearly_seasonality= False)
prophet.fit(model_data[['Date', 'Confirmed']].rename({'Date':'ds', 'Confirmed': 'y'}, axis = 1))

future = prophet.make_future_dataframe(periods=21, freq='d')
forecast = prophet.predict(future)
fig1 = prophet.plot_components(forecast)
act_preds = model_data[['Date', 'Confirmed']].rename({'Date':'ds', 'Confirmed': 'y'}, axis = 1).merge(forecast[['ds', 'yhat']], how = 'left', on = 'ds')

# Curve Fitting

In [ ]:
import scipy

model_data = base_data[(base_data['Date'] >= pd.Timestamp("2020-04-08 00:00:00")) & (base_data['Date'] <= pd.Timestamp("2020-07-10 00:00:00"))]
table = model_data.pivot_table(index = 'Date', columns = 'State', values = 'Confirmed', aggfunc="sum")
table.drop(['LD', 'State Unassigned'], axis = 1, inplace = True)

table.reset_index(inplace = True)
min_day = table['Date'].dt.dayofyear.min()
table['day_offset'] = table['Date'].dt.dayofyear - min_day + 1

xtest = base_data[base_data['Date'] > pd.Timestamp("2020-07-10 00:00:00")]
test_table = model_data.pivot_table(index = 'Date', columns = 'State', values = 'Confirmed', aggfunc="sum")
test_table.drop(['LD', 'State Unassigned'], axis = 1, inplace = True)

test_table.reset_index(inplace = True)
test_table['day_offset'] = test_table['Date'].dt.dayofyear - min_day + 1

for state in states:
  xdata = table[['day_offset', state]]
  x = xdata['day_offset'].values
  y = xdata[state].values
  eqn = lambda t,a,b: a*np.exp(b*t)
  opt = scipy.optimize.curve_fit(eqn,  x,  y,  p0=(4, 0.1))
  a0 = opt[0][0]
  b0 = opt[0][1]
  print(f"{state}: {a0}*exp({b0}*t)")
  test_table = test_table[['day_offset', state]]
  eqn = lambda t: a0*np.exp(b0*t)j
  test_table['pred'] = test_table['day_offset'].apply(eqn)
  test_table['pred'] = test_table['pred'].astype(int)
  display(test_table.set_index('day_offset').plot())

Maharashtra: 516.5525599453657*exp(0.027902933934057357*t)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Tamil Nadu: 167.45722622481577*exp(0.035868992699898974*t)


KeyError: ignored

In [ ]:
# table.plot()#.write_html(os.getcwd() + '/Plots.html')

In [ ]:
# dir(table.plot())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
dist_states = []

for state in states:
  print(state)
  model_data = base_data[base_data['State'] == state].drop('State', axis = 1)
  model_data = model_data[(model_data['Date'] >= pd.Timestamp("2020-04-08 00:00:00")) & (model_data['Date'] <= pd.Timestamp("2020-06-28 00:00:00"))]
  sin1 = model_data.set_index('Date')[['Confirmed']]
  dist_states = dist_states + [model_data.set_index('Date')['Confirmed']]

  # X = model_data.set_index('Date')['Confirmed'].values

  # model0 = ARIMA(X, dates=None,order=(2,1,0))
  # model1 = model0.fit(disp=1)

  # decomposition = seasonal_decompose(X.reshape(len(X),),freq=100)
  # ### insert your data seasonality in 'freq'

  # trend = decomposition.trend
  # seasonal = decomposition.seasonal
  # residual = decomposition.resid
  # # plt.plot(sin1,label=state)

  # # plt.legend(loc=2)
  # # plt.show()

Maharashtra
Tamil Nadu
Delhi
Karnataka
Andhra Pradesh
Uttar Pradesh
West Bengal
Gujarat
Telangana
Rajasthan
Bihar
Assam
Haryana
Madhya Pradesh
Odisha
Kerala
Jammu and Kashmir
Punjab
Jharkhand
Chhattisgarh
Uttarakhand
Goa
Tripura
Puducherry
Manipur
Himachal Pradesh
Ladakh
Nagaland
Arunachal Pradesh
Dadra and Nagar Haveli and Daman and Diu
Chandigarh
Meghalaya
Sikkim
Mizoram
Andaman and Nicobar Islands


In [ ]:
pd.concat(pred_df, axis = 1)

,Maharashtra,Tamil Nadu,Delhi,Karnataka,Andhra Pradesh,Uttar Pradesh,West Bengal,Gujarat,Telangana,Rajasthan,Bihar,Assam,Haryana,Madhya Pradesh,Odisha,Kerala,Jammu and Kashmir,Punjab,Jharkhand,Chhattisgarh,Uttarakhand,Goa,Tripura,Puducherry,Manipur,Himachal Pradesh,Ladakh,Nagaland,Arunachal Pradesh,Dadra and Nagar Haveli and Daman and Diu,Chandigarh,Meghalaya,Sikkim,Mizoram,Andaman and Nicobar Islands
0,7401,3746,2750,766,873,731,398,727,950,286,261,400,409,131,245,156,123,160,40,98,59,79,2,0,52,13,76,44,475,10,3,4,-2,63,0
1,5348,4101,3118,1140,878,678,568,598,1049,349,265,315,438,207,206,148,144,126,27,71,78,76,22,37,69,18,60,28,54,14,6,1,0,13,6
2,7531,4055,3135,1163,981,709,494,624,1040,332,260,350,425,171,247,132,155,155,51,64,69,72,35,85,79,20,67,18,-746,25,3,2,0,33,6
3,6194,4263,3489,921,978,719,594,618,1139,318,268,350,352,201,237,166,163,100,26,95,65,81,26,40,79,21,96,16,-1468,25,6,2,0,3,0
4,8470,4308,3485,1059,1050,775,541,627,1180,336,268,345,440,168,250,137,178,159,68,70,78,77,36,59,92,19,95,18,1549,23,6,1,4,23,12
5,6784,4364,3794,1204,1142,738,597,701,1292,358,285,381,362,189,251,161,179,85,38,77,76,90,37,128,66,20,84,14,4198,28,8,1,3,11,-6
6,9630,4646,3899,1205,1203,767,572,642,1411,364,289,363,389,163,259,154,208,153,72,94,74,81,45,31,53,23,110,15,1877,31,7,1,3,6,16
7,7093,4510,4069,1106,1287,780,581,641,1508,374,270,361,358,189,265,153,176,86,44,65,66,83,47,-1,57,28,75,20,-11673,34,6,1,2,23,-8
8,10193,4811,4403,1387,1357,796,573,635,1640,388,275,368,405,185,254,170,165,128,85,69,92,82,48,97,41,25,88,24,-15404,47,8,2,3,4,15
9,7090,4839,4653,1248,1500,790,582,638,1713,388,284,381,322,198,282,157,198,90,36,66,79,83,57,132,56,29,84,31,11083,43,4,1,3,17,-4


In [ ]:
pd.concat(conf_df[0:35], axis = 1)

,Maharashtra,Tamil Nadu,Delhi,Karnataka,Andhra Pradesh,Uttar Pradesh,West Bengal,Gujarat,Telangana,Rajasthan,Bihar,Assam,Haryana,Madhya Pradesh,Odisha,Kerala,Jammu and Kashmir,Punjab,Jharkhand,Chhattisgarh,Uttarakhand,Goa,Tripura,Puducherry,Manipur,Himachal Pradesh,Ladakh,Nagaland,Arunachal Pradesh,Dadra and Nagar Haveli and Daman and Diu,Chandigarh,Meghalaya,Sikkim,Mizoram,Andaman and Nicobar Islands
0,5257,3949,2084,1105,793,681,624,626,975,389,394,302,381,184,245,122,144,202,62,101,8,53,34,42,42,26,1,36,5,15,3,1,0,0,7
1,4878,3943,2199,947,704,664,652,620,945,348,370,613,338,223,206,131,260,150,64,63,50,64,8,24,7,11,9,8,4,10,6,2,0,9,7
2,5537,3882,2442,1272,657,564,611,675,1018,304,216,548,393,268,251,151,198,100,35,82,66,72,8,25,26,26,17,42,4,16,6,2,1,0,3
3,6328,4343,2373,1502,845,769,649,681,1213,350,478,479,568,245,229,160,154,116,60,73,37,95,39,63,19,35,1,34,37,25,4,3,12,2,9
4,6364,4329,2520,1694,837,972,669,687,1892,390,426,365,494,191,561,211,170,153,112,52,64,94,93,22,37,19,11,4,20,50,4,4,1,0,7
5,7074,4280,2505,1839,765,757,743,712,1850,480,349,1202,545,307,495,240,227,172,42,96,45,108,25,80,9,13,3,24,7,37,5,8,1,2,3
6,6555,4150,2244,1925,998,1153,895,725,1590,632,403,735,457,326,469,225,183,174,68,46,31,77,23,42,41,17,0,27,10,32,7,2,22,22,6
7,5368,3827,1379,1843,1322,929,861,735,1831,524,280,786,499,354,456,193,246,208,47,98,37,52,111,65,24,14,0,35,1,27,21,16,0,11,16
8,5134,3616,2008,1498,1178,1332,850,778,1879,716,385,814,495,343,571,272,256,258,164,110,69,90,24,32,40,6,36,19,6,27,5,6,2,1,6
9,6603,3756,2033,2062,1062,1188,986,783,1924,659,749,696,691,409,527,301,330,158,116,111,28,136,57,110,5,18,6,13,11,15,15,6,6,5,2


In [ ]:
xdata = base_data[(base_data['Date'] >= pd.Timestamp("2020-06-29 00:00:00")) & (base_data['Date'] <= pd.Timestamp("2020-07-19 00:00:00"))][['Date', 'State', 'Confirmed']]
for state in states:
  model_data = xdata[xdata['State'] == state].drop('State', axis = 1)
  conf_df = conf_df + [model_data[['Confirmed']].rename({'Confirmed':state}, axis = 1)]